In [1]:
import os 
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import re

In [2]:
property = gpd.read_file('data/Market_value_usable_hedonic_data.csv')

In [3]:
property.head()

,field_1,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,FullStreetNumber,RoadName,suburb_loc,YearSold,AgeAtSale,GrossSalePrice,...,RnkIMDNoHo,RnkIMDNoHe,RnkIMDNoEd,RnkIMDNoAc,DECILE_high,DECILE_prim,Median_Income,CBD_DIST,Ratio,z_score
0,0,0,0,0,4,Romanee Lane,Wigram,2024,18,920000,...,1516.0,1356.0,1004.0,1313.0,7.0,9.0,92781,6615.87185063722,0.6630434782608695,0.8634969826688191
1,1,2,2,2,52,Sioux Avenue,Wigram,2024,18,670000,...,2136.0,2283.0,2205.0,2014.0,8.0,3.0,92781,6657.46003773288,1.0,0.2064113830650419
2,2,3,3,3,34,Contrail Street,Wigram,2024,18,750000,...,1139.0,1104.0,1035.0,940.0,7.0,7.0,92781,4921.376982828,1.04,0.4166787749382506
3,3,4,4,4,34,Contrail Street,Wigram,2018,13,550000,...,1139.0,1104.0,1035.0,940.0,7.0,7.0,92781,4921.376982828,0.9454545454545454,0.1089897047447711
4,4,6,6,6,38,Keene Street,Wigram,2024,18,795000,...,1618.0,1677.0,1357.0,1468.0,8.0,7.0,92781,5923.28314260674,0.981132075471698,0.5349541828669305


In [4]:
property.shape

(59601, 39)

In [5]:
property_2018 = property[property['YearSold'] == '2018']
property_2018.shape

(6213, 39)

In [6]:
property_2018_gdf = gpd.GeoDataFrame(
  property_2018,
  geometry=gpd.points_from_xy(
    property_2018['gd2000co'], property_2018['gd2000_yco']
  ),
  crs='EPSG:4326'
)

In [15]:
pd.set_option('display.float_format', str)

In [16]:
pd.to_numeric(property_2018_gdf['GrossSalePrice']).describe()

count               6213.0
mean     501225.7052953485
std     210831.21803570833
min                68750.0
25%               373400.0
50%               451500.0
75%               573000.0
max              1725000.0
Name: GrossSalePrice, dtype: float64

In [4]:
property_cleaned = property.loc[:,~property.columns.str.contains('^Unnamed')]

In [5]:
data_2024 = property_cleaned[property_cleaned['YearSold'] == 2024]

In [6]:
pp_gdf = gpd.GeoDataFrame(
  property_cleaned,
  geometry=gpd.points_from_xy(
    property_cleaned['gd2000co'], property_cleaned['gd2000_yco']
  ),
  crs='EPSG:4326'
)

In [7]:
pp_gdf = pp_gdf.to_crs(2193)

In [8]:
sa2 = gpd.read_file('data/chc-boundaries/sa2/statistical-area-2-2023-generalised.shp')
ta = gpd.read_file('data/chc-boundaries/territorial-authority-2021-generalised.gpkg')
sa1 = gpd.read_file('data/chc-boundaries/statistical-area-1-2025.gpkg')

In [9]:
ta_chc = ta[ta['TA2021_V1_00_NAME_ASCII'].str.lower().str.contains('christchurch')]
ta_chc.head()

,TA2021_V1_00,TA2021_V1_00_NAME,TA2021_V1_00_NAME_ASCII,LAND_AREA_SQ_KM,AREA_SQ_KM,Shape_Length,geometry
51,060,Christchurch City,Christchurch City,1415.437559,1608.602046,323655.640667,"MULTIPOLYGON (((1575709.746 5195732.032, 15757..."


In [10]:
sa2_chc = sa2.clip(ta_chc)

In [11]:
sa1_chc = sa1.clip(ta_chc)

In [12]:
pp_sa1 = gpd.sjoin(
  pp_gdf,
  sa1[['SA12025_V1_00', 'geometry']],
  how='left',
  predicate='within'
)

In [13]:
pp_sa1.head()

,field_1,FullStreetNumber,RoadName,suburb_loc,YearSold,AgeAtSale,GrossSalePrice,CapitalValue,LandValue,ImprovementsValue,...,RnkIMDNoAc,DECILE_high,DECILE_prim,Median_Income,CBD_DIST,Ratio,z_score,geometry,index_right,SA12025_V1_00
0,0,4,Romanee Lane,Wigram,2024,18,920000,610000,230000,380000,...,1313.0,7.0,9.0,92781,6615.87185063722,0.6630434782608695,0.8634969826688191,POINT (1563525.529 5176721.7),32034,7035777
1,1,52,Sioux Avenue,Wigram,2024,18,670000,670000,385000,285000,...,2014.0,8.0,3.0,92781,6657.46003773288,1.0,0.2064113830650419,POINT (1563284.861 5177812.598),24095,7024757
2,2,34,Contrail Street,Wigram,2024,18,750000,780000,405000,375000,...,940.0,7.0,7.0,92781,4921.376982828,1.04,0.4166787749382506,POINT (1564765.959 5177559.301),32049,7035849
3,3,34,Contrail Street,Wigram,2018,13,550000,520000,195000,325000,...,940.0,7.0,7.0,92781,4921.376982828,0.9454545454545454,0.1089897047447711,POINT (1564765.959 5177559.301),32049,7035849
4,4,38,Keene Street,Wigram,2024,18,795000,780000,405000,375000,...,1468.0,8.0,7.0,92781,5923.28314260674,0.981132075471698,0.5349541828669305,POINT (1563911.214 5177663.664),4305,7024850


In [14]:
pp_sa1.columns

Index(['field_1', 'FullStreetNumber', 'RoadName', 'suburb_loc', 'YearSold',
       'AgeAtSale', 'GrossSalePrice', 'CapitalValue', 'LandValue',
       'ImprovementsValue', 'PriceValueRelationship', 'LandArea',
       'TotalFloorArea', 'gd2000co', 'gd2000_yco', 'cycleways_DIST',
       'cycle_DENS', 'on_DIST', 'on_DENS', 'Green_DIST', 'water_DIST',
       'bus_DIST', 'Census_Pop', 'RnkIMDNoEm', 'RnkIMDNoIn', 'RnkIMDNoCr',
       'RnkIMDNoHo', 'RnkIMDNoHe', 'RnkIMDNoEd', 'RnkIMDNoAc', 'DECILE_high',
       'DECILE_prim', 'Median_Income', 'CBD_DIST', 'Ratio', 'z_score',
       'geometry', 'index_right', 'SA12025_V1_00'],
      dtype='object')

In [15]:
pd.set_option('display.max_columns', None)

In [16]:
pp_sa1 = pp_sa1.drop(columns=['index_right'])

In [17]:
# pp_sa1_2024 = pp_sa1[pp_sa1['YearSold'] == '2024']
pp_sa1_2024 = pp_sa1.copy()

In [18]:
pp_sa1_2024.head()

,field_1,FullStreetNumber,RoadName,suburb_loc,YearSold,AgeAtSale,GrossSalePrice,CapitalValue,LandValue,ImprovementsValue,PriceValueRelationship,LandArea,TotalFloorArea,gd2000co,gd2000_yco,cycleways_DIST,cycle_DENS,on_DIST,on_DENS,Green_DIST,water_DIST,bus_DIST,Census_Pop,RnkIMDNoEm,RnkIMDNoIn,RnkIMDNoCr,RnkIMDNoHo,RnkIMDNoHe,RnkIMDNoEd,RnkIMDNoAc,DECILE_high,DECILE_prim,Median_Income,CBD_DIST,Ratio,z_score,geometry,SA12025_V1_00
0,0,4,Romanee Lane,Wigram,2024,18,920000,610000,230000,380000,Market - Index,503.0,180.0,172.54837185,-43.56156975,438.2566223,3.0,139.090744,3718.7,28.0225598384,380.0050427426,147.6790926072,522.0,1899.0,980.0,1124.0,1516.0,1356.0,1004.0,1313.0,7.0,9.0,92781,6615.87185063722,0.6630434782608695,0.8634969826688191,POINT (1563525.529 5176721.7),7035777
1,1,52,Sioux Avenue,Wigram,2024,18,670000,670000,385000,285000,Market - Index,255.0,108.0,172.54546583,-43.55173557,1377.9197998,0.0,40.0,2705.1000000000004,45.0830301477,346.8356161025,365.5694706714,684.0,2233.0,1796.0,1261.0,2136.0,2283.0,2205.0,2014.0,8.0,3.0,92781,6657.46003773288,1.0,0.2064113830650419,POINT (1563284.861 5177812.598),7024757
2,2,34,Contrail Street,Wigram,2024,18,750000,780000,405000,375000,Market - Index,314.0,152.0,172.56378487,-43.55408767,1259.1204834,0.0,105.9322739,4007.400000000001,12.435140072,152.4231878411,876.2078786712001,792.0,1373.0,999.0,541.0,1139.0,1104.0,1035.0,940.0,7.0,7.0,92781,4921.376982828,1.04,0.4166787749382506,POINT (1564765.959 5177559.301),7035849
3,3,34,Contrail Street,Wigram,2018,13,550000,520000,195000,325000,Market - Index,314.0,152.0,172.56378487,-43.55408767,1259.1204834,0.0,105.9322739,4007.400000000001,12.435140072,152.4231878411,876.2078786712001,792.0,1373.0,999.0,541.0,1139.0,1104.0,1035.0,940.0,7.0,7.0,92781,4921.376982828,0.9454545454545454,0.1089897047447711,POINT (1564765.959 5177559.301),7035849
4,4,38,Keene Street,Wigram,2024,18,795000,780000,405000,375000,Market - Index,337.0,141.0,172.55320995,-43.55310713,1050.9067383,0.0,220.818634,3633.8,55.7873791783,504.8540565003,119.6799940051,573.0,1702.0,1821.0,669.0,1618.0,1677.0,1357.0,1468.0,8.0,7.0,92781,5923.28314260674,0.981132075471698,0.5349541828669305,POINT (1563911.214 5177663.664),7024850


In [19]:
pp_sa1_2024['GrossSalePrice'] = (
  pp_sa1_2024['GrossSalePrice'].astype(str).str.replace(',', '', regex=False).str.strip().astype(float)
)

In [20]:
pp_sa1_stats = pp_sa1_2024.groupby('SA12025_V1_00').agg(
  median_price=('GrossSalePrice', 'median'),
  mean_price=('GrossSalePrice', 'mean'),
  sale_count=('GrossSalePrice', 'count'),
)

In [21]:
pp_sa1_stats.head(30)

,median_price,mean_price,sale_count
SA12025_V1_00,,,
7024281,520000.0,5.200000e+05,1
7024286,630000.0,6.639571e+05,35
7024288,559000.0,5.673973e+05,25
7024294,980000.0,9.800000e+05,2
7024297,930000.0,1.175000e+06,3
7024298,767000.0,8.487500e+05,8
7024299,920250.0,8.959167e+05,6
7024300,656000.0,7.193553e+05,19
7024301,755000.0,8.131789e+05,19


In [22]:
sa1_prices = sa1.merge(pp_sa1_stats,on='SA12025_V1_00', how='left')

In [23]:
sa1_prices.head()

,SA12025_V1_00,LANDWATER,LANDWATER_NAME,LAND_AREA_SQ_KM,AREA_SQ_KM,SHAPE_Length,geometry,median_price,mean_price,sale_count
0,7000000,12,Mainland,285.483500,285.483500,192821.759465,"MULTIPOLYGON (((1586902.537 6191293.705, 15869...",NaN,NaN,NaN
1,7000002,22,Inlet,0.000000,63.689388,165389.978313,"MULTIPOLYGON (((1590891.908 6186235.518, 15909...",NaN,NaN,NaN
2,7000004,12,Mainland,168.746325,168.746325,74393.214746,"MULTIPOLYGON (((1599412.256 6149874.568, 15994...",NaN,NaN,NaN
3,7000005,12,Mainland,26.031531,26.031531,27130.387293,"MULTIPOLYGON (((1608719.037 6151274.842, 16087...",NaN,NaN,NaN
4,7000006,12,Mainland,51.238250,51.238250,63061.661174,"MULTIPOLYGON (((1614246.453 6154525.888, 16142...",NaN,NaN,NaN


In [24]:
sa1_prices_suburb = gpd.sjoin(
  sa1_prices,
  sa2_chc[['SA22023__2', 'geometry']],
  how='left',
  predicate='intersects'
)

In [25]:
sa1_prices_suburb.head()

,SA12025_V1_00,LANDWATER,LANDWATER_NAME,LAND_AREA_SQ_KM,AREA_SQ_KM,SHAPE_Length,geometry,median_price,mean_price,sale_count,index_right,SA22023__2
0,7000000,12,Mainland,285.483500,285.483500,192821.759465,"MULTIPOLYGON (((1586902.537 6191293.705, 15869...",NaN,NaN,NaN,NaN,NaN
1,7000002,22,Inlet,0.000000,63.689388,165389.978313,"MULTIPOLYGON (((1590891.908 6186235.518, 15909...",NaN,NaN,NaN,NaN,NaN
2,7000004,12,Mainland,168.746325,168.746325,74393.214746,"MULTIPOLYGON (((1599412.256 6149874.568, 15994...",NaN,NaN,NaN,NaN,NaN
3,7000005,12,Mainland,26.031531,26.031531,27130.387293,"MULTIPOLYGON (((1608719.037 6151274.842, 16087...",NaN,NaN,NaN,NaN,NaN
4,7000006,12,Mainland,51.238250,51.238250,63061.661174,"MULTIPOLYGON (((1614246.453 6154525.888, 16142...",NaN,NaN,NaN,NaN,NaN


In [26]:
pp_riccarton = sa1_prices_suburb[sa1_prices_suburb['SA22023__2'].fillna('').str.lower().str.contains('riccarton')]

In [27]:
pp_riccarton.head()

,SA12025_V1_00,LANDWATER,LANDWATER_NAME,LAND_AREA_SQ_KM,AREA_SQ_KM,SHAPE_Length,geometry,median_price,mean_price,sale_count,index_right,SA22023__2
4281,7024733,12,Mainland,0.043396,0.043396,1025.690108,"MULTIPOLYGON (((1563464.163 5179601.804, 15634...",405000.0,427728.542857,35.0,71.0,Riccarton Racecourse
4317,7024972,12,Mainland,0.067690,0.067690,1943.807060,"MULTIPOLYGON (((1565211.672 5180107.753, 15653...",467500.0,477125.000000,8.0,140.0,Upper Riccarton
4346,7025257,12,Mainland,0.017582,0.017582,737.150523,"MULTIPOLYGON (((1567385.088 5179838.002, 15673...",431000.0,487727.272727,11.0,38.0,Riccarton West
4346,7025257,12,Mainland,0.017582,0.017582,737.150523,"MULTIPOLYGON (((1567385.088 5179838.002, 15673...",431000.0,487727.272727,11.0,161.0,Riccarton South
4347,7025258,12,Mainland,0.024424,0.024424,645.192612,"MULTIPOLYGON (((1567129.89 5180093.028, 156711...",367500.0,442624.000000,17.0,38.0,Riccarton West


In [29]:
m = pp_riccarton.explore(
    column="median_price",           # what you want to map
    cmap="Reds",                     # continuous red gradient
    legend=True,                     # add colourbar
    tiles="CartoDB positron",        # clean basemap
    tooltip=["SA12025_V1_00",        # your SA1 code column
             "median_price",
             "sale_count"],
    style_kwds={
        "weight": 0.3,
        "opacity": 0.8,
        "color": "black"
    },
    missing_kwds={
        "color": "lightgrey",
        "label": "No data",
        "hatch": "///"
    },
)

m

In [ ]:
m = sa1_prices_suburb.explore(
    column="median_price",           # what you want to map
    cmap="Reds",                     # continuous red gradient
    legend=True,                     # add colourbar
    tiles="CartoDB positron",        # clean basemap
    tooltip=["SA12025_V1_00",        # your SA1 code column
             "median_price",
             "sale_count"],
    style_kwds={
        "weight": 0.3,
        "opacity": 0.8,
        "color": "black"
    },
    missing_kwds={
        "color": "lightgrey",
        "label": "No data",
        "hatch": "///"
    },
    height=650
)

m